### An update of df's with train, validation and test sets (preapred by Sathiesh)
######  2020.07.13  

###### ver. 2020.08.10

The files prepared by Sathiesh:
- ixi_test.csv
- test.csv
- train_val.
- nfbs_test.csv

These files holds paths to the images on DGX server, updated paths to the coresponding files on Titan server are added to the same csv files. New files are attributed with the suffix: _\*_mk_3d.csv_

**Folder:** 2.2_train_valid_test_sets


At the end of the notebook a function that checks whether all files pointed by a paths in df's exist is run.

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
import pandas as pd

from PIL import Image
from pathlib import Path
from fastai.utils.mem import *

In [4]:
import sys
sys.path.append('functions')

#%aimport functions00, functions01

from functions00 import *
from functions01 import *

In [5]:
mk_get_host_info()

**********************************************************************************************************************************
Settings:
	HOST:  mmiv-ml-titan
	PATH_ROOT_DATA:  /data-10tb/shared/skull/train-3d-iso
	PATH_GIT_HUB:  /data-10tb/marek/github_codes/skull-stripping-1/fastai

3D DFs paths (_mk_3D):
	IXI_TEST_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/ixi_test_mk_3d.csv
	TEST_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/test_mk_3d.csv
	TRAIN_VAL_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/train_val_mk_3d.csv
	NFBS_TEST_3D : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/nfbs_test_mk_3d.csv

Error files (pickle):
	ERROR_FILES : /data-10tb/marek/github_codes/skull-stripping-1/fastai/2.2_train_valid_test_sets/error_files
****************************************************************************************************

In [6]:
# Input are files prepared by Shathiesh
file1 = 'train_val.csv'
file2 = 'test.csv'
file3 = 'ixi_test.csv'
file4 = 'nfbs_test.csv'

file = file4


data_folder = Path('2.2_train_valid_test_sets')
df = pd.read_csv(data_folder/file)

In [7]:
print(df.shape)
df.head(2)

(125, 9)


,path,age_at_scan,gender,subject_id,img_id,source,path_bias,brain_mask_bias_nn,brain_mask_bias_lin
0,NaN,99.0,U,A00028185,A00028185,NFBS,/data-external/sathiesh/br...,/data-external/sathiesh/br...,NaN
1,NaN,99.0,U,A00028352,A00028352,NFBS,/data-external/sathiesh/br...,/data-external/sathiesh/br...,NaN


#### Rename some colum names (add suffix 'dgx')

In [8]:
df.rename(columns = {'path':'t1_pth_dgx'}, inplace = True)
df.rename(columns = {'path_bias':'bias_pth_dgx'}, inplace = True)
df.rename(columns = {'brain_mask_bias_nn':'mask_nn_pth_dgx'}, inplace = True)
df.rename(columns = {'brain_mask_bias_lin':'mask_lin_pth_dgx'}, inplace = True)
df.head(2)

,t1_pth_dgx,age_at_scan,gender,subject_id,img_id,source,bias_pth_dgx,mask_nn_pth_dgx,mask_lin_pth_dgx
0,NaN,99.0,U,A00028185,A00028185,NFBS,/data-external/sathiesh/br...,/data-external/sathiesh/br...,NaN
1,NaN,99.0,U,A00028352,A00028352,NFBS,/data-external/sathiesh/br...,/data-external/sathiesh/br...,NaN


In [9]:
# prepare new fields to a df
# updated mask names to *_iso_nn.nii.gz

###################################################
# ###  ASSUMPTION: WE ARE WORKING ON TITAN!!! ### #
###################################################

if 'titan' in HOST:
    root1 = PATH_ROOT_DATA
    t1_pth_titan = []
    bias_pth_titan = []
    mask_nn_pth_titan = []
    mask_lin_pth_titan = []


    #print(root1)
    
    # get pahts from t1 or bias images (if not t1 present)
    srcpth = df.t1_pth_dgx if isinstance(df.t1_pth_dgx[0], str) else df.bias_pth_dgx
    for p in srcpth:
        # update Sathiesh's path to my path
        p1= p.split('brains/')[1]
        path_t1 = root1/p1
        if not path_t1.exists():
            print(f'{path_t1}', end='\r')
        t1_pth_titan.append(str(path_t1))

        # mask_nn
        msk_nn = str(path_t1).split('_T1_')[0] + '_T1_biascorr_brain_mask_iso_nn.nii.gz'
        path_mask_nn = root1/msk_nn
        if not path_mask_nn.exists():
            print(f'{p} {path_mask_nn}', end='\r')
        mask_nn_pth_titan.append(path_mask_nn)

        # mask_lin
        msk_lin = str(path_t1).split('_T1_')[0] + '_T1_biascorr_brain_mask_iso_lin.nii.gz'
        path_mask_lin = root1/msk_lin
        if not path_mask_lin.exists():
            print(f'{p} {path_mask_lin}', end='\r')
        mask_lin_pth_titan.append(path_mask_lin)

        # T1_biascorr
        bias = str(path_t1).split('_T1_')[0] + '_T1_biascorr_iso.nii.gz'
        path_bias = root1/bias
        if not path_bias.exists():
            print(f'{p} {path_bias}', end='\r')
        bias_pth_titan.append(path_bias)
else:
    print('WARNING\tThis operanion MUST be run on Tinan!!!!')

In [9]:
print(len(t1_pth_titan), len(bias_pth_titan), len(mask_nn_pth_titan), len(mask_lin_pth_titan) )

125 125 125 125


In [10]:
# update valies in a 'path' column
# df = df.assign(path=t1_pth)
# Add to the end of df
df['t1_pth_titan'] = t1_pth_titan
df['bias_pth_titan'] = bias_pth_titan
df['mask_nn_pth_titan'] = mask_nn_pth_titan
df['mask_lin_pth_titan'] = mask_lin_pth_titan

df.head(3).T

,0,1,2
t1_pth_dgx,NaN,NaN,NaN
age_at_scan,99,99,99
gender,U,U,U
subject_id,A00028185,A00028352,A00032875
img_id,A00028185,A00028352,A00032875
source,NFBS,NFBS,NFBS
bias_pth_dgx,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-external/sathiesh/br...
mask_nn_pth_dgx,/data-external/sathiesh/br...,/data-external/sathiesh/br...,/data-external/sathiesh/br...
mask_lin_pth_dgx,NaN,NaN,NaN
t1_pth_titan,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...


#### Reorder columns

In [11]:
cols = df.columns.tolist()
print(cols)

cols2 = []
for k in ['t1_pth_dgx','bias_pth_dgx', 'mask_nn_pth_dgx', 'mask_lin_pth_dgx','t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan', 'mask_lin_pth_titan']:
    if k in cols:
        cols2.append(k)
        cols.remove(k)
cols2.extend(cols)  

df = df[cols2]
print(cols2)
df.head(2)

['t1_pth_dgx', 'age_at_scan', 'gender', 'subject_id', 'img_id', 'source', 'bias_pth_dgx', 'mask_nn_pth_dgx', 'mask_lin_pth_dgx', 't1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan', 'mask_lin_pth_titan']
['t1_pth_dgx', 'bias_pth_dgx', 'mask_nn_pth_dgx', 'mask_lin_pth_dgx', 't1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan', 'mask_lin_pth_titan', 'age_at_scan', 'gender', 'subject_id', 'img_id', 'source']


,t1_pth_dgx,bias_pth_dgx,mask_nn_pth_dgx,mask_lin_pth_dgx,t1_pth_titan,bias_pth_titan,mask_nn_pth_titan,mask_lin_pth_titan,age_at_scan,gender,subject_id,img_id,source
0,NaN,/data-external/sathiesh/br...,/data-external/sathiesh/br...,NaN,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,99.0,U,A00028185,A00028185,NFBS
1,NaN,/data-external/sathiesh/br...,/data-external/sathiesh/br...,NaN,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...,99.0,U,A00028352,A00028352,NFBS


#### Save table as a csv to 2.2_train_valid_test_sets folder

In [10]:
# SAVE CAN BE DONE ONLY ON TITAN
if 'titan' in HOST:
    save = 0

    # host = titan or dgx
    #host = 'titan' if 'titan' in HOST else 'dgx'

    if save:
        fn = file.split('.')
        fsave = f'{fn[0]}_mk_3d.' + fn[1]

        df.to_csv(data_folder/fsave, index=False)
else:
    print('WARNING\tThis operanion MUST be run on Tinan!!!!')

#### Load df from a file if NOT exists

In [6]:
if 'df' in locals() or 'df' in globals():
    print('You have defined a df')
else:
    file = Path('2.2_train_valid_test_sets/train_val_mk_3d.csv')
    print(f'Data Frame from disc:\n\t{file}')
    df = pd.read_csv(file)
df.head(2).T

Data Frame from disc:
	2.2_train_valid_test_sets/train_val_mk_3d.csv


,0,1
t1_pth_dgx,/data-external/sathiesh/br...,/data-external/sathiesh/br...
bias_pth_dgx,/data-external/sathiesh/br...,/data-external/sathiesh/br...
mask_nn_pth_dgx,/data-external/sathiesh/br...,/data-external/sathiesh/br...
mask_lin_pth_dgx,/data-external/sathiesh/br...,/data-external/sathiesh/br...
t1_pth_titan,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...
bias_pth_titan,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...
mask_nn_pth_titan,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...
mask_lin_pth_titan,/data-10tb/shared/skull/tr...,/data-10tb/shared/skull/tr...
age_at_scan,74.3,78.5
gender,M,F


#### Check if all files from df exists - TITAN

In [7]:
if 'titan' in HOST:
    #df_files_exists(df, ['t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan', 'mask_lin_pth_titan'])
    mk_df_files_exists(df, ['t1_pth_titan', 'bias_pth_titan', 'mask_nn_pth_titan'])

Looing for NOT EXISTING files in "df.t1_pth_titan"
	All files are OK

Looing for NOT EXISTING files in "df.bias_pth_titan"
	All files are OK

Looing for NOT EXISTING files in "df.mask_nn_pth_titan"
	All files are OK



#### Check if all files from df exists - DGX

In [109]:
if 'dgx'.upper() in HOST:
    mk_df_files_exists(df, ['t1_pth_dgx', 'bias_pth_dgx', 'mask_nn_pth_dgx', 'mask_lin_pth_dgx'])